```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Perpendicular random fields

This notebook is used to test the generation of perpendicular random fields produced on the GPU. Hipp hurra!

## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import OceanStateNoise, GPUDrifterCollection


In [ ]:
%cuda_context_handler gpu_ctx
gpu_stream = cuda.Stream()

In [ ]:
def imshow(im, interpolation="None", title=None, figsize=(4,4)):
    fig = plt.figure(figsize=figsize)
    plt.imshow(im, interpolation=interpolation, origin='lower')
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
        
def imshow2(xi, nu, interpolation="None", title=None):
    fig, axs = plt.subplots(1,2, figsize=(8,4))
    
    xi_im = axs[0].imshow(xi, interpolation=interpolation, origin='lower')
    axs[0].set_title("xi")
    plt.colorbar(xi_im, ax=axs[0])
    
   
    nu_im = axs[1].imshow(nu, interpolation=interpolation, origin='lower')
    axs[1].set_title("nu")
    plt.colorbar(nu_im, ax=axs[1])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
     
def imshow3(eta, hu, hv, interpolation="None", title=None, interior=False):
    fig, axs = plt.subplots(1,3, figsize=(12,4))
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower')
    axs[0].set_title("eta")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower')
    axs[1].set_title("hu")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower')
    axs[2].set_title("hv")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()

def imshow6(eta1, hu1, hv1, eta2, hu2, hv2, interpolation="None", title=None):
    fig, axs = plt.subplots(3,3, figsize=(12,13))
    
    range_eta = max(np.max(np.abs(eta1)), np.max(np.abs(eta2)))
    range_huv = max(np.max(np.abs(hu1)), np.max(np.abs(hu2)), np.max(np.abs(hv1)), np.max(np.abs(hv2)))
    
    eta_im = axs[0,0].imshow(eta1, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[0,0].set_title("eta 1")
    plt.colorbar(eta_im, ax=axs[0,0])
    
    hu_im = axs[0,1].imshow(hu1, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[0,1].set_title("hu 1")
    plt.colorbar(hu_im, ax=axs[0,1])

    hv_im = axs[0,2].imshow(hv1, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[0,2].set_title("hv 1")
    plt.colorbar(hv_im, ax=axs[0,2])

    
    eta_im2 = axs[1,0].imshow(eta2, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[1,0].set_title("eta 2")
    plt.colorbar(eta_im, ax=axs[1,0])
    
    hu_im2 = axs[1,1].imshow(hu2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1,1].set_title("hu 2")
    plt.colorbar(hu_im, ax=axs[1,1])

    hv_im2 = axs[1,2].imshow(hv2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1,2].set_title("hv 2")
    plt.colorbar(hv_im, ax=axs[1,2])
    
    
    eta_im3 = axs[2,0].imshow(eta1 - eta2, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[2,0].set_title("diff eta")
    plt.colorbar(eta_im, ax=axs[2,0])
    
    hu_im3 = axs[2,1].imshow(hu1 - hu2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2,1].set_title("diff hu")
    plt.colorbar(hu_im, ax=axs[2,1])

    hv_im3 = axs[2,2].imshow(hv1 - hv2, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2,2].set_title("diff hv")
    plt.colorbar(hv_im, ax=axs[2,2])

    if title is not None:
        plt.suptitle(title, fontsize=18)
    plt.tight_layout()   

# Testing 


In [ ]:
if 'newnoise' in globals():
    newnoise.cleanUp()
reload(OceanStateNoise)
nx, ny = 210, 210
dx, dy = 100.0, 100.0
interpolation_factor = 7

noise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, \
                                        nx, ny, dx, dy, \
                                        Common.BoundaryConditions(2,2,2,2),
                                        False,
                                        interpolation_factor=interpolation_factor)

xi_norm, nu_norm, xi_nu_dot = noise.generatePerpendicularNormalDistributions()
print("xi_norm, nu_norm, xi_nu_dot: ", (xi_norm, nu_norm, xi_nu_dot))

xi = noise.getRandomNumbers()
nu = noise.getPerpendicularRandomNumbers()

print("xi.shape: ", xi.shape)
print("nu.shape: ", nu.shape)

imshow2(xi, nu)

xiTnu = np.sum(xi*nu)
xiTxi = np.sum(xi*xi)
nuTnu = np.sum(nu*nu)
print("xiTnu: ", xiTnu)
print("nuTnu: ", nuTnu)
print("xiTxi: ", xiTxi)

noise.getDoubleNormAndDot()
reduction_buffer_host = noise.getReductionBuffer()
print("xi_norm, nu_norm, xi_nu_dot: ", reduction_buffer_host)
      